<a href="https://colab.research.google.com/github/ProfAaronBaird/HealthAnalytics/blob/main/notebooks/Health_Analytics_Synthea_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Getting ready for Health Analytics

**Playing around with getting Synthea data into Colab**



In [1]:
import pandas as pd


In [2]:
# Don't wrap repr(DataFrame) across additional lines
pd.set_option("display.expand_frame_repr", False)

# Set max rows displayed in output to 25
pd.set_option("display.max_rows", 50)

In [3]:
# Load data from GitHub (clone first, then use)
!git clone https://github.com/ProfAaronBaird/HealthAnalytics.git

fatal: destination path 'HealthAnalytics' already exists and is not an empty directory.


In [4]:
# Load data
path_root = "HealthAnalytics/data/synthea/"
path_patients_csv = 'csv_ga_1500_patients/patients.csv'
path_encounters_csv = 'csv_ga_1500_patients/encounters.csv'
dfp = pd.read_csv(path_root + path_patients_csv)
dfe = pd.read_csv(path_root + path_encounters_csv)

In [5]:
dfp.head

<bound method NDFrame.head of                                         Id   BIRTHDATE   DEATHDATE          SSN    DRIVERS    PASSPORT PREFIX        FIRST            LAST SUFFIX      MAIDEN MARITAL   RACE    ETHNICITY GENDER                   BIRTHPLACE                       ADDRESS          CITY    STATE           COUNTY      ZIP        LAT        LON  HEALTHCARE_EXPENSES  HEALTHCARE_COVERAGE
0     89e54b28-92b3-a1ec-a03c-96b514364706  2013-10-04         NaN  999-88-4785        NaN         NaN    NaN    Nestor901         Kihn564    NaN         NaN     NaN  white  nonhispanic      M   Sandy Springs  Georgia  US          690 Gorczany Pathway          Rome  Georgia     Floyd County  30165.0  34.258053 -85.190860            171485.04              1698.70
1     462ac5df-b85d-cd8a-8ab9-d3b30de21c4f  2010-03-13         NaN  999-60-7195        NaN         NaN    NaN  Gregorio366       Spinka232    NaN         NaN     NaN  black  nonhispanic      M           Redan  Georgia  US           396 Toy 

In [6]:
dfe.head

<bound method NDFrame.head of                                          Id                 START                  STOP                               PATIENT                          ORGANIZATION                              PROVIDER                                 PAYER ENCOUNTERCLASS       CODE                                 DESCRIPTION  BASE_ENCOUNTER_COST  TOTAL_CLAIM_COST  PAYER_COVERAGE   REASONCODE           REASONDESCRIPTION
0      8395790e-e899-2f04-4580-dd2412044fa8  2011-05-21T22:19:41Z  2011-05-21T22:34:41Z  462ac5df-b85d-cd8a-8ab9-d3b30de21c4f  ca5a8d47-a974-3e1c-b988-139caa0f0ffc  dbc3172d-6a8f-3558-90c1-0550e46944ff  42c4fca7-f8a9-3cd1-982a-dd9751bf3e2a       wellness  410620009                Well child visit (procedure)               102.15            102.15          102.15          NaN                         NaN
1      651a70ab-a7ec-d8ec-1a4d-38d0bba42d8b  2014-01-04T20:35:39Z  2014-01-04T20:50:39Z  b1551fe9-5158-35d0-e030-a9d6cd4aab6c  7bd55b9c-8e40-3ae4-90ab-1b3aad1

In [7]:
dfp.columns

Index(['Id', 'BIRTHDATE', 'DEATHDATE', 'SSN', 'DRIVERS', 'PASSPORT', 'PREFIX',
       'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'MARITAL', 'RACE', 'ETHNICITY',
       'GENDER', 'BIRTHPLACE', 'ADDRESS', 'CITY', 'STATE', 'COUNTY', 'ZIP',
       'LAT', 'LON', 'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE'],
      dtype='object')

In [8]:
dfe.columns

Index(['Id', 'START', 'STOP', 'PATIENT', 'ORGANIZATION', 'PROVIDER', 'PAYER',
       'ENCOUNTERCLASS', 'CODE', 'DESCRIPTION', 'BASE_ENCOUNTER_COST',
       'TOTAL_CLAIM_COST', 'PAYER_COVERAGE', 'REASONCODE',
       'REASONDESCRIPTION'],
      dtype='object')

In [9]:
# Join the encounter data with the patient data
dfj = dfe.set_index('Id').join(dfp.set_index('Id'))

In [10]:
dfj.head

<bound method NDFrame.head of                                                      START                  STOP                               PATIENT                          ORGANIZATION                              PROVIDER                                 PAYER ENCOUNTERCLASS       CODE                                 DESCRIPTION  BASE_ENCOUNTER_COST  TOTAL_CLAIM_COST  PAYER_COVERAGE   REASONCODE           REASONDESCRIPTION BIRTHDATE DEATHDATE  SSN DRIVERS PASSPORT PREFIX FIRST LAST SUFFIX MAIDEN MARITAL RACE ETHNICITY GENDER BIRTHPLACE ADDRESS CITY STATE COUNTY  ZIP  LAT  LON  HEALTHCARE_EXPENSES  HEALTHCARE_COVERAGE
Id                                                                                                                                                                                                                                                                                                                                                                                   

In [11]:
# Group by patient ID and sum the claim cost 
dfjg = dfj.groupby('PATIENT')['TOTAL_CLAIM_COST'].sum().reset_index()
dfjg.sort_values(by='TOTAL_CLAIM_COST',axis=0,ascending=False)

,PATIENT,TOTAL_CLAIM_COST
1625,f2e74f69-b052-4fc7-d337-e3a83b2f3e15,568134.28
986,92ed000b-32de-5bec-c7dd-3b3222c2c79a,503710.05
112,115d4a6d-f02e-3f5e-9645-f8b3cab91ebf,370484.77
471,4811cc8c-ade1-e268-0a7b-6315b5ac037c,326103.63
892,85c912ce-cf3f-266f-76c6-732a516322fc,303558.20
...,...,...
1197,b30b69bd-9083-1cd9-dd43-cb1f86fc8887,204.30
308,2ddfa06e-4905-93aa-32bd-fbf7e6db9ae0,204.30
290,2b172bdd-3f98-5294-0e24-dcf78cbf855d,204.30
586,5a036bc2-62a0-09be-6eb3-348bcaf770b0,102.15


In [12]:
dfjg.describe()

,TOTAL_CLAIM_COST
count,1719.000000
mean,8499.441611
std,28796.127177
min,102.150000
25%,1685.475000
50%,2880.630000
75%,6220.935000
max,568134.280000


In [13]:
print(dfjg)

                                   PATIENT  TOTAL_CLAIM_COST
0     000160d7-7b1f-4e53-905f-8f2162eecd1e           5904.27
1     00040dbb-e986-3543-ed9c-a0a9dce46e1a           1368.81
2     001db349-fdbd-7786-5a08-5fcb82f0a159          18153.83
3     003b9ff8-28a1-fcfb-9652-8b1e02ab6f69           2145.15
4     0071a6ae-d3ae-0ce1-0f3c-c54bea8eb855          15430.74
...                                    ...               ...
1714  fef16a73-a65b-ba74-0871-e8964a61c90d           1164.51
1715  ff325141-eefb-d944-26ab-f46e9cf31546          10174.14
1716  ff5bec9a-5e93-5092-6e6f-d5895ec1306b           3370.95
1717  ff79b5d3-af4f-6d31-e0be-47ab558703a6           3003.21
1718  ff999738-5c1f-5657-e307-a6552b6ab943           3391.38

[1719 rows x 2 columns]


In [14]:
# Top 10 most expensive patients
dfjg.nlargest(n=10,columns=['TOTAL_CLAIM_COST'])

,PATIENT,TOTAL_CLAIM_COST
1625,f2e74f69-b052-4fc7-d337-e3a83b2f3e15,568134.28
986,92ed000b-32de-5bec-c7dd-3b3222c2c79a,503710.05
112,115d4a6d-f02e-3f5e-9645-f8b3cab91ebf,370484.77
471,4811cc8c-ade1-e268-0a7b-6315b5ac037c,326103.63
892,85c912ce-cf3f-266f-76c6-732a516322fc,303558.20
598,5bccfc71-673b-ab13-ae93-bc390b1bb1b8,294866.34
806,7afb5aa9-245b-27e2-dce6-2604add985d3,279693.98
245,240b3e65-28a1-b377-fb5c-4b8a70c7ed3d,194800.40
120,12214d49-26c0-068d-9c02-069d819c88f7,192346.62
317,2f7bc766-64cf-ed3d-59c8-fb5cdc445799,164821.73


In [27]:
# Info on most expensive patient
dfe.loc[dfe['PATIENT'] == 'd19ed2e8-7eda-0bef-be7f-d8c9257f4198']

,Id,START,STOP,PATIENT,ORGANIZATION,PROVIDER,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION


In [16]:
# Get top 10%
dfjg['decile'] = pd.qcut(dfjg['TOTAL_CLAIM_COST'], q=10, labels=False, precision=0)

In [17]:
print(dfjg.sort_values(by='decile',axis=0,ascending=False))

                                   PATIENT  TOTAL_CLAIM_COST  decile
1268  be4e394a-c266-4623-1f52-842a8b3eb823          28234.26       9
1522  e5ac558d-0a0e-d844-1546-5f5df7ac2abc          41118.41       9
620   5ea65410-a1bf-a5d3-aeb5-c8dbd8884f1a          26559.93       9
960   8f8d0c57-6442-8b77-bb18-d7127e9bc6f3          17312.65       9
968   909d0ea8-c0d8-ecf4-866f-e1cc2b3e1cc4          26183.42       9
...                                    ...               ...     ...
1575  eb7fa2d4-387a-7031-d14b-ed614345074d            633.33       0
295   2bccc864-c414-e1af-63d5-49757ff0f70b            776.34       0
1391  d04a727e-2841-dfaa-6476-84f73a710e88            980.64       0
298   2c384c49-eeaa-55e2-43a7-f72ffd728552           1164.51       0
329   31b65a77-bbc2-2e10-ab3c-2b859f6a10a8            429.03       0

[1719 rows x 3 columns]


In [18]:
dfjg.loc[dfjg['PATIENT'] == 'd19ed2e8-7eda-0bef-be7f-d8c9257f4198']

,PATIENT,TOTAL_CLAIM_COST,decile


In [19]:
# Get ready for supervised learning... classify column for users in the top 10% of cost (utilization)
dfjg['risk_highest'] = 0
dfjg.loc[dfjg.decile == 9, 'risk_highest'] = 1

dfjg['risk_rising'] = 0
dfjg.loc[dfjg.decile.isin([7, 8]), 'risk_rising'] = 1

dfjg['risk_moderate'] = 0
dfjg.loc[dfjg.decile.isin([4, 5, 6]), 'risk_moderate'] = 1

dfjg['risk_low'] = 0
dfjg.loc[dfjg.decile.isin([0, 1, 2, 3]), 'risk_low'] = 1

In [20]:
print(dfjg)

                                   PATIENT  TOTAL_CLAIM_COST  decile  risk_highest  risk_rising  risk_moderate  risk_low
0     000160d7-7b1f-4e53-905f-8f2162eecd1e           5904.27       7             0            1              0         0
1     00040dbb-e986-3543-ed9c-a0a9dce46e1a           1368.81       1             0            0              0         1
2     001db349-fdbd-7786-5a08-5fcb82f0a159          18153.83       9             1            0              0         0
3     003b9ff8-28a1-fcfb-9652-8b1e02ab6f69           2145.15       3             0            0              0         1
4     0071a6ae-d3ae-0ce1-0f3c-c54bea8eb855          15430.74       9             1            0              0         0
...                                    ...               ...     ...           ...          ...            ...       ...
1714  fef16a73-a65b-ba74-0871-e8964a61c90d           1164.51       0             0            0              0         1
1715  ff325141-eefb-d944-26ab-f4

In [31]:
dfp.columns

Index(['Id', 'BIRTHDATE', 'DEATHDATE', 'SSN', 'DRIVERS', 'PASSPORT', 'PREFIX',
       'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'MARITAL', 'RACE', 'ETHNICITY',
       'GENDER', 'BIRTHPLACE', 'ADDRESS', 'CITY', 'STATE', 'COUNTY', 'ZIP',
       'LAT', 'LON', 'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE'],
      dtype='object')

In [30]:
dfjg.columns

Index(['PATIENT', 'TOTAL_CLAIM_COST', 'decile', 'risk_highest', 'risk_rising',
       'risk_moderate', 'risk_low'],
      dtype='object')

In [22]:
# Merge back with patient dataframe
dfm = pd.merge(dfp, dfjg, left_index=True, right_index=True) # or left_on='Id', right_on='PATIENT'

In [28]:
dfm.describe()

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,PATIENT,TOTAL_CLAIM_COST,decile,risk_highest,risk_rising,risk_moderate,risk_low
0,89e54b28-92b3-a1ec-a03c-96b514364706,2013-10-04,NaN,999-88-4785,NaN,NaN,NaN,Nestor901,Kihn564,NaN,NaN,NaN,white,nonhispanic,M,Sandy Springs Georgia US,690 Gorczany Pathway,Rome,Georgia,Floyd County,30165.0,34.258053,-85.190860,171485.04,1698.70,000160d7-7b1f-4e53-905f-8f2162eecd1e,5904.27,7,0,1,0,0
1,462ac5df-b85d-cd8a-8ab9-d3b30de21c4f,2010-03-13,NaN,999-60-7195,NaN,NaN,NaN,Gregorio366,Spinka232,NaN,NaN,NaN,black,nonhispanic,M,Redan Georgia US,396 Toy Path Apt 31,Fairburn,Georgia,Fulton County,30291.0,33.621166,-84.557903,293303.31,1940.85,00040dbb-e986-3543-ed9c-a0a9dce46e1a,1368.81,1,0,0,0,1
2,88f35580-4bf1-a9ff-53f0-67d6a1c84967,2010-07-17,NaN,999-60-6136,NaN,NaN,NaN,Lilly908,Heidenreich818,NaN,NaN,NaN,white,hispanic,F,Sandy Springs Georgia US,519 Brown Arcade,Rome,Georgia,Floyd County,30161.0,34.199195,-85.127983,219650.11,1963.00,001db349-fdbd-7786-5a08-5fcb82f0a159,18153.83,9,1,0,0,0
3,b1551fe9-5158-35d0-e030-a9d6cd4aab6c,2014-01-04,NaN,999-45-2270,NaN,NaN,NaN,Bernardo699,Hoeger474,NaN,NaN,NaN,black,nonhispanic,M,Cordele Georgia US,888 Stehr Arcade,Johns Creek,Georgia,Fulton County,30024.0,34.086427,-84.066621,149932.64,1745.15,003b9ff8-28a1-fcfb-9652-8b1e02ab6f69,2145.15,3,0,0,0,1
4,ff79b5d3-af4f-6d31-e0be-47ab558703a6,1999-05-29,NaN,999-94-6081,S99920240,X10549853X,Ms.,Lonna614,Marks830,NaN,NaN,NaN,black,nonhispanic,F,East Point Georgia US,401 Ryan Rest Unit 30,Atlanta,Georgia,Fulton County,30354.0,33.789270,-84.385336,431636.27,2198.04,0071a6ae-d3ae-0ce1-0f3c-c54bea8eb855,15430.74,9,1,0,0,0


In [29]:
dfm.head()

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,PATIENT,TOTAL_CLAIM_COST,decile,risk_highest,risk_rising,risk_moderate,risk_low
0,89e54b28-92b3-a1ec-a03c-96b514364706,2013-10-04,NaN,999-88-4785,NaN,NaN,NaN,Nestor901,Kihn564,NaN,NaN,NaN,white,nonhispanic,M,Sandy Springs Georgia US,690 Gorczany Pathway,Rome,Georgia,Floyd County,30165.0,34.258053,-85.190860,171485.04,1698.70,000160d7-7b1f-4e53-905f-8f2162eecd1e,5904.27,7,0,1,0,0
1,462ac5df-b85d-cd8a-8ab9-d3b30de21c4f,2010-03-13,NaN,999-60-7195,NaN,NaN,NaN,Gregorio366,Spinka232,NaN,NaN,NaN,black,nonhispanic,M,Redan Georgia US,396 Toy Path Apt 31,Fairburn,Georgia,Fulton County,30291.0,33.621166,-84.557903,293303.31,1940.85,00040dbb-e986-3543-ed9c-a0a9dce46e1a,1368.81,1,0,0,0,1
2,88f35580-4bf1-a9ff-53f0-67d6a1c84967,2010-07-17,NaN,999-60-6136,NaN,NaN,NaN,Lilly908,Heidenreich818,NaN,NaN,NaN,white,hispanic,F,Sandy Springs Georgia US,519 Brown Arcade,Rome,Georgia,Floyd County,30161.0,34.199195,-85.127983,219650.11,1963.00,001db349-fdbd-7786-5a08-5fcb82f0a159,18153.83,9,1,0,0,0
3,b1551fe9-5158-35d0-e030-a9d6cd4aab6c,2014-01-04,NaN,999-45-2270,NaN,NaN,NaN,Bernardo699,Hoeger474,NaN,NaN,NaN,black,nonhispanic,M,Cordele Georgia US,888 Stehr Arcade,Johns Creek,Georgia,Fulton County,30024.0,34.086427,-84.066621,149932.64,1745.15,003b9ff8-28a1-fcfb-9652-8b1e02ab6f69,2145.15,3,0,0,0,1
4,ff79b5d3-af4f-6d31-e0be-47ab558703a6,1999-05-29,NaN,999-94-6081,S99920240,X10549853X,Ms.,Lonna614,Marks830,NaN,NaN,NaN,black,nonhispanic,F,East Point Georgia US,401 Ryan Rest Unit 30,Atlanta,Georgia,Fulton County,30354.0,33.789270,-84.385336,431636.27,2198.04,0071a6ae-d3ae-0ce1-0f3c-c54bea8eb855,15430.74,9,1,0,0,0


New things learned: 
Adding groupby sum column to the main dataframe: use transform:
df['Data4'] = df['Data3'].groupby(df['Date']).transform('sum')


Ideas: All under the heading "Risk Stratification":
- For high_util, might actually want four columns, risk_highest, risk_risk, risk_moderate, risk_low
- Might also want another way to categorize risk, number of conditions and see if correlated with costs: https://www.nachc.org/wp-content/uploads/2019/03/Risk-Stratification-Action-Guide-Mar-2019.pdf

**Playing around with R code in-line with Python**

In [24]:
%load_ext rpy2.ipython

In [25]:
%%R
x <- seq(0, 2*pi, length.out=50)
x

 [1] 0.0000000 0.1282283 0.2564565 0.3846848 0.5129131 0.6411414 0.7693696
 [8] 0.8975979 1.0258262 1.1540544 1.2822827 1.4105110 1.5387393 1.6669675
[15] 1.7951958 1.9234241 2.0516523 2.1798806 2.3081089 2.4363372 2.5645654
[22] 2.6927937 2.8210220 2.9492502 3.0774785 3.2057068 3.3339351 3.4621633
[29] 3.5903916 3.7186199 3.8468481 3.9750764 4.1033047 4.2315330 4.3597612
[36] 4.4879895 4.6162178 4.7444460 4.8726743 5.0009026 5.1291309 5.2573591
[43] 5.3855874 5.5138157 5.6420439 5.7702722 5.8985005 6.0267288 6.1549570
[50] 6.2831853
